Welcome to Tortoise! 🐢🐢🐢🐢

Before you begin, I **strongly** recommend you turn on a GPU runtime.

There's a reason this is called "Tortoise" - this model takes up to a minute to perform inference for a single sentence on a GPU. Expect waits on the order of hours on a CPU.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Imports used through the rest of the notebook.
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F

import IPython

from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio, load_voice, load_voices

# This will download all the models used by Tortoise from the HF hub.
# tts = TextToSpeech()
# If you want to use deepspeed the pass use_deepspeed=True nearly 2x faster than normal
tts = TextToSpeech(kv_cache=True)

/Users/fazavyalo2/personal_git/tortoise-tts-senschat/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at jbetker/wav2vec2-large-robust-ft-libritts-voxpopuli were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_v', 'wav2vec2.encoder.pos_conv_embed.conv.weight_g']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some 

In [3]:
tts.autoregressive_batch_size = 4

In [11]:
# This is the text that will be spoken.
text = [
    "Hello my name is",
    "Joining two modalities"
]
# text = "Joining two modalities"
# Here's something for the poetically inclined.. (set text=)
"""
Then took the other, as just as fair,
And having perhaps the better claim,
Because it was grassy and wanted wear;
Though as for that the passing there
Had worn them really about the same,"""

# Pick a "preset mode" to determine quality. Options: {"ultra_fast", "fast" (default), "standard", "high_quality"}. See docs in api.py
preset = "ultra_fast"

In [12]:
CUSTOM_VOICE_NAME = "custom"

In [13]:
voice_samples, conditioning_latents = load_voice(CUSTOM_VOICE_NAME)

In [21]:
gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents,
                          preset=preset, num_autoregressive_samples=8)

Generating autoregressive samples..


100%|██████████| 2/2 [00:21<00:00, 10.56s/it]


Computing best candidates using CLVP


100%|██████████| 2/2 [00:03<00:00,  1.96s/it]


Transforming autoregressive outputs into audio..


100%|██████████| 10/10 [00:03<00:00,  2.64it/s]


In [22]:
torchaudio.save(f'generated-{CUSTOM_VOICE_NAME}.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio(f'generated-{CUSTOM_VOICE_NAME}.wav')

In [8]:
for i in range(len(gen)):
    torchaudio.save(f'generated-{CUSTOM_VOICE_NAME}_{i}.wav', gen[i].squeeze(0).cpu(), 24000)
    IPython.display.Audio(f'generated-{CUSTOM_VOICE_NAME}_{i}.wav')

In [9]:
IPython.display.Audio(f'generated-{CUSTOM_VOICE_NAME}_0.wav')

In [10]:
IPython.display.Audio(f'generated-{CUSTOM_VOICE_NAME}_1.wav')